# Loop for running bnpstep to fit existing FIONA traces

### Import necessary packages

In [1]:
# All inputs
import numpy as np
import matplotlib.pyplot as plt
import pickle
from pathlib import Path
import tkinter as tk
from tkinter import filedialog
import scipy.io
import os
import bnpstep as bnp

def insert_before_pattern(fname, insert_str, pattern):
    idx = fname.find(pattern)
    if idx == -1:
        return fname  # return unchanged
    return fname[:idx] + insert_str + fname[idx:]

### Select (multiple) .mat fiona files for loading

In [16]:
# Select .mat fiona files for loading

# Hide the main tkinter window
root = tk.Tk()
root.withdraw()

# Open file dialog to select multiple files
file_paths = filedialog.askopenfilenames(
    title='Select .mat files',
    filetypes=[('MAT files', '*.mat')]
)

# file_paths is a tuple of strings (paths)
print("Number of files: ", len(file_paths))
print(list(file_paths))  # Convert to list if needed


Number of files:  25
['/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/PO4 5000/250415-M19-000508_31341_fiona_wblip.mat', '/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/PO4 5000/250415-M19-000508_38881_fiona_wblip.mat', '/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/PO4 5000/250415-M21-012255_4973_fiona_wblip.mat', '/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/PO4 5000/250415-M21-012255_27370_fiona_wblip.mat', '/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/PO4 5000/250415-M21-012255_38733_fiona_wblip.mat', '/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/PO4 5000/250415-M21-012255_44361_fiona_wblip.mat', '/Users/slivka/Documents/University of California/Research/Dip comparison

### Run through all files and save bnpstep results in .mat and .pkl format

Note: We will read the step number in the trace to give us an estimate of number of steps. It might slightly increase time, but it is necessary to make sure we aren't overloading the matrices. It seems by parameter sweep that 10000 iterations is good to get most of the convergence by eye.

In [17]:
# now run and save all results to load in later for the trace long-axis. (Short-axis to come later)

for i in range(len(file_paths)):
    matdata = scipy.io.loadmat(file_paths[i], squeeze_me=False, struct_as_record=True)
    p = Path(file_paths[i])

    # print(matdata['data'].dtype)
    data_struct = matdata['data'][0, 0]  # Extract the struct instance
    # for name in matdata['data'].dtype.names:
    #     field = data_struct[name]
    #     print(f"{name}: type = {type(field)}, shape = {getattr(field, 'shape', None)}")

    # for name in matdata['data'].dtype.names:
    #     print(name, data_struct[name])

    # Write a csv to load data, then immediately delete
    t_trace_arr = np.array([data_struct['time'].reshape(len(data_struct['time']),1).flatten(), data_struct['trace'][:,0].reshape(len(data_struct['time']),1).flatten()])
    # t_trace_arr = t_trace_arr.reshape(len(data_struct['time']),2)
    t_trace_arr = t_trace_arr.transpose()
    print(t_trace_arr.shape)
    np.savetxt(str(p.parent)+'/tempdata.csv', t_trace_arr, delimiter=',')

    # Initialize the data with the sampler with about twice as many steps as the user found
    A = data_struct['trace'][:,2]
    estimated_steps = np.sum(np.roll(A,1)!=A)
    if estimated_steps > 128/2:
        print('skipping', p.stem, 'for too many steps')
    else:
        sampler = bnp.BNPStep(B_max=int(2*estimated_steps))
        sampler.load_data(str(p.parent)+'/tempdata.csv', has_timepoints=True)

        # if os.path.exists('tests/tempdata.csv'):
        #     os.remove('tests/tempdata.csv')
        #     print("File deleted.")
        # else:
        #     print("File not found.")

        # Now find the steps
        print("Sampler initialized with max steps:", int(2*estimated_steps), "sampling for 5000 iterations.")
        sampler.analyze(num_samples=5000)
        sampler.results_to_file(outfile=str(p.parent)+'/'+p.stem+'_trial_sample')
        # sampler.visualize_results(plot_type='step')

        # change the trace fit after we did the new fitting
        new_steps = sampler.export_step_data()
        data_struct['trace'][:,2] = np.reshape(new_steps, data_struct['trace'][:,2].shape)

        # Now build the dictionary again and rewrite it to the new filename
        # Build the dictionary dynamically
        new_filename = insert_before_pattern(file_paths[i], "_bnpstep", "_fiona")
        rewritematdata = {'data': {}}

        for name in matdata['data'].dtype.names:
            rewritematdata['data'][name] = data_struct[name]

        scipy.io.savemat(new_filename, rewritematdata)
    

(4654, 2)
Sampler initialized with max steps: 34 sampling for 5000 iterations.


/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:344: RuntimeWarning: divide by zero encountered in log
  new_log_likelihood = ((num_data / 2) * np.log(eta_vec[-1] / (2 * np.pi))) - exponent_term
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:393: RuntimeWarning: divide by zero encountered in log
  prior_eta = ((phi - 1) * np.log(eta_vec[-1])) - ((phi * eta_vec[-1]) / eta_ref) - np.log(sp.special.gamma(phi)) - \
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:399: RuntimeWarning: invalid value encountered in scalar add
  log_posterior = log_likelihood + prior_b_m + prior_h_m + prior_t_m + prior_eta + prior_f
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:250: RuntimeWarning: divide by zero encountered in scalar divide
  acceptance_ratio = (1 / (np.exp(exponent_prop + exponent_old))) ** (1 / temp)
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:250: RuntimeWarning: overflow encountered in exp
  acceptance_ratio = (1 / (np.exp(exponent_prop + exponent_old))

(4340, 2)
Sampler initialized with max steps: 40 sampling for 5000 iterations.


/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:344: RuntimeWarning: divide by zero encountered in log
  new_log_likelihood = ((num_data / 2) * np.log(eta_vec[-1] / (2 * np.pi))) - exponent_term
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:393: RuntimeWarning: divide by zero encountered in log
  prior_eta = ((phi - 1) * np.log(eta_vec[-1])) - ((phi * eta_vec[-1]) / eta_ref) - np.log(sp.special.gamma(phi)) - \
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:399: RuntimeWarning: invalid value encountered in scalar add
  log_posterior = log_likelihood + prior_b_m + prior_h_m + prior_t_m + prior_eta + prior_f
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:250: RuntimeWarning: overflow encountered in exp
  acceptance_ratio = (1 / (np.exp(exponent_prop + exponent_old))) ** (1 / temp)


(2541, 2)
Sampler initialized with max steps: 24 sampling for 5000 iterations.


...

(2502, 2)
Sampler initialized with max steps: 50 sampling for 5000 iterations.


/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:250: RuntimeWarning: overflow encountered in exp
  acceptance_ratio = (1 / (np.exp(exponent_prop + exponent_old))) ** (1 / temp)


(1206, 2)
Sampler initialized with max steps: 34 sampling for 5000 iterations.


/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:344: RuntimeWarning: divide by zero encountered in log
  new_log_likelihood = ((num_data / 2) * np.log(eta_vec[-1] / (2 * np.pi))) - exponent_term
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:393: RuntimeWarning: divide by zero encountered in log
  prior_eta = ((phi - 1) * np.log(eta_vec[-1])) - ((phi * eta_vec[-1]) / eta_ref) - np.log(sp.special.gamma(phi)) - \
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:399: RuntimeWarning: invalid value encountered in scalar add
  log_posterior = log_likelihood + prior_b_m + prior_h_m + prior_t_m + prior_eta + prior_f
/Users/slivka/Documents/GitHub/bnp-step/bnpsampler.py:250: RuntimeWarning: overflow encountered in exp
  acceptance_ratio = (1 / (np.exp(exponent_prop + exponent_old))) ** (1 / temp)


(23310, 2)
skipping 250416-M1-163752_18194_fiona_wblip_wblip for too many steps


## Visualization of data with python (Not operational with current setup)

In [13]:
rootdir1 = '/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/D1848E 1000/'
rootdir2 = '/Users/slivka/Documents/University of California/Research/Dip comparison temp folder/bnpstep trial/D1848E 5000/'
sampler = bnp.BNPStep(B_max=122)
sampler.load_data(str(p.parent)+'/tempdata.csv', has_timepoints=True)
sampler.results_from_file(filename=rootdir1+'250413-M9-134855_10397_fiona_wblip_trial_sample')
sampler.visualize_results(plot_type='step')

ValueError: Dataset is empty, cannot generate graphs.